In [101]:
import cv2
from yellowbrick.cluster import KElbowVisualizer
import pandas as pd 
import numpy as np
from color_util import *
from PIL import Image
from sklearn.cluster import KMeans
from IPython.display import display
import matplotlib.pyplot as plt
import time
from color_cluster import *
from os import path

data=pd.read_csv("pat_time_count_nyc_sample.csv",header=0,sep="^")

In [31]:
start=pd.to_datetime('2020/4')
end=pd.to_datetime('2020/6')
x=data[pd.to_datetime(data.time)>start].copy()
x=x[pd.to_datetime(data.time)<end]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [7]:
len(data.pattern_id.unique())

1841

In [17]:
patterns=(x.pattern_id.unique())

In [40]:
def df_image_exist(x,folder=None):
    patterns=(x.pattern_id.unique())
    rm_list=[]
    for p in patterns:
        filepath=folder+str(p)+'.jpg'
        if not path.exists(filepath):
            rm_list.append(p)
    x=x[~x['pattern_id'].isin(rm_list)]
    return x

In [41]:
filepath="/Users/yajinghuang/Desktop/insight/project_ideas/marcc/colors/crop_image/"
x=df_image_exist(x,folder=filepath)

In [46]:
len(x[x.time=='2020/05'].pattern_id.unique())

1200

In [112]:

def color_pattern_clustering(pattern_id,folder="crop_image/",outfolder="cluster_dict/"):
    filepath=folder+str(pattern_id)+'.jpg'
    outfile=outfolder+str(pattern_id)+'.txt'
    if path.exists(filepath) and (not path.exists(outfile)):
        image=cv2.imread(filepath)
        image=prepare_image(image)
        color=kmeans_op(image)
        np.savetxt(outfile,color)
        print(pattern_id,"write to dict")
    else:
        print('image not exists, or txt already exists')
    return None
        
def get_color_data_from_clusters(df,month):
    output={}
    df = df[df.time==month]
    weights=[]
    colors = np.empty((0,4), int)
    color_dict={}
    for index, row in df.iterrows():
        pattern_id=row['pattern_id']
        if pattern_id not in color_dict.keys():
            fname='cluster_dict/'+str(pattern_id)+'.txt'
            color_dict[pattern_id]=np.loadtxt(fname)
            
        color =color_dict[pattern_id]
        color[:,4]=color[:,4]*row.count
        colors = np.append(colors, color, axis=0)
    output[month:colors]
    
    return output

def color_vs_month(df):
    color_dict_months={}
    months=df.time.unique()
    for m in months:
        color_dict_months.update(get_color_data_from_clusters(df,m))
    return color_dict_months

def df_pattern_to_color(df):
    ids=df.pattern_id.unique()
    for i in ids:
        color_pattern_clustering(i)
    print('df pattern to color done')
    

def get_df_sample(df,frac=0.1):
    months=df.time.unique()
    df_out=df[df.time==months[0]].sample(frac=frac)
    for m in months[1:]:
        d=df[df.time==m].sample(frac = frac)
        if not d.empty:
            df_out=df_out.append(d,ignore_index=True)
    return df_out




In [113]:
#pipeline
data=pd.read_csv("pat_time_count_nyc_sample.csv",header=0,sep="^")
sample=get_df_sample(data,frac=0.05)
df_pattern_to_color(sample)
color_dict_months=color_vs_month(sample)

In [115]:
sample=get_df_sample(data,frac=0.05)

In [83]:
colors = np.append(colors, x, axis=0)

In [85]:
colors.shape

(8, 4)

In [117]:
x={'a':3,'b':4}
x.update({})